In [1]:
from utils.binaries import *
from utils.plotting import *
from utils.Auger.SD.UubRandoms import UubRandom, filter_and_downsample

09:48:00 [INFO] utils          -- ( +2855ms) -- import logging
09:48:00 [INFO] utils.binaries -- (   +98ms) -- import numpy as np
09:48:00 [INFO] utils.binaries -- (    +9ms) -- import uncertainties
09:48:00 [INFO] utils.binaries -- (  +170ms) -- import pandas as pd
09:48:00 [INFO] utils.binaries -- (    +2ms) -- import binaries.tools as tools
09:48:01 [INFO] utils.plotting -- (  +280ms) -- import matplotlib.pyplot as plt
09:48:01 [INFO] utils.plotting -- (  +331ms) -- import seaborn as so
09:48:01 [DBUG] utils.plotting -- (    +6ms) -- font size set to 9.5
09:48:01 [DBUG] utils.plotting -- (    +1ms) -- label size set to 13.0
09:48:01 [DBUG] utils.plotting -- (    +1ms) -- figure size set to [6.6, 3.3]
09:48:01 [DBUG] utils.plotting -- (    +1ms) -- markersize set to 2.0
09:48:01 [DBUG] utils.plotting -- (    +1ms) -- usetex set to False
09:48:01 [INFO] utils.plotting -- (    +2ms) -- import plotting.tools as plot


In [2]:
def time_over_threshold(
    traces: np.ndarray, threshold: float = 0.2, multiplicity: int = 12
) -> bool:

    pmt_multiplicity_check = lambda sums: sum(sums > multiplicity) > 1

    first_120_bins = traces[:, :120]
    pmt_running_sum = (first_120_bins >= threshold).sum(axis=1)

    for i in range(120, traces.shape[1] + 1):
        if pmt_multiplicity_check(pmt_running_sum):
            return True  # check multiplicity for each window
        if i == traces.shape[1]:
            return False  # we've reached the end of the trace

        new_over_threshold = np.array(traces[:, i] > threshold, dtype=int)
        old_over_threshold = np.array(traces[:, i - 120] > threshold, dtype=int)
        pmt_running_sum += new_over_threshold - old_over_threshold


# def time_over_threshold(traces : np.ndarray, threshold : float = 0.2, multiplicity : int = 12) -> bool :

#     windows = np.lib.stride_tricks.sliding_window_view(traces, (3, 120))[0]
#     for pmt1, pmt2, pmt3 in windows:

#         pmt1_active = (pmt1 > threshold).sum() >= multiplicity
#         pmt2_active = (pmt2 > threshold).sum() >= multiplicity
#         pmt3_active = (pmt3 > threshold).sum() >= multiplicity

#         if sum([pmt1_active, pmt2_active, pmt3_active]) > 1:
#             return True
#     else: return False


def time_over_threshold_deconvoluted(
    traces: np.ndarray, threshold: float = 0.2, multiplicity: int = 12
) -> bool:

    # for information on this see GAP note 2018-01
    dt = 25  # UB bin width
    tau = 67  # decay constant
    decay = np.exp(-dt / tau)  # decay term
    deconvoluted_trace = []

    for pmt in traces:
        deconvoluted_pmt = [
            (pmt[i] - pmt[i - 1] * decay) / (1 - decay) for i in range(1, len(pmt))
        ]
        deconvoluted_trace.append(deconvoluted_pmt)

    return time_over_threshold(np.array(deconvoluted_trace), threshold, multiplicity)

In [ ]:
from itertools import product

# 300 scanning points
multiplicities = range(2, 14)
threshold = np.linspace(0.01, 0.25, 25)
params = list(product(multiplicities, threshold))

m, t = params[int(sys.argv[2])]
tot_sum, totd_sum = 0, 0
total_time = 0

for File in tools.ProgressBar(UubRandom("NuriaJr", "wcd"), newline=False):
    traces = File["traces"]
    vem_peak = File["vem_peak"]

    for trace, vem in zip(traces, vem_peak):
        filter_and_downsampled = filter_and_downsample(*trace)
        calibrated = filter_and_downsampled / vem[:, np.newaxis]

        tot_sum += time_over_threshold(calibrated, t, m)
        totd_sum += time_over_threshold_deconvoluted(calibrated, t, m)
        total_time += 2048 * 8.33e-9

with open(f"/cr/users/filip/Data/totRateMap/{STATION}.txt", "a+") as file:
    file.write(f"{m} {t} {tot_sum} {totd_sum} {total_time}\n")